In [1]:
import pymc as pm
import pandas as pd
import arviz as az
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("iris_filtered.csv")

# Extract the last column as a NumPy array
data = df.iloc[:, -1].to_numpy()

# PyMC Model
with pm.Model() as model:
    theta = pm.Beta("theta", alpha=1, beta=1)
    obs = pm.Bernoulli("obs", p=theta, observed=data)
    trace = pm.sample(1000, return_inferencedata=True)

# Summarizing posterior
az.plot_trace(trace)
plt.show()

summary = az.summary(trace)
print(summary)

# Decision over the posterior
rope = [0.45, 0.55]
hdi = az.hdi(trace["posterior"], hdi_prob=0.94)["theta"]
decision = "Within ROPE" if rope[0] <= hdi[0] and hdi[1] <= rope[1] else "Outside ROPE"
print(f"Decision based on ROPE and HDI: {decision}")

# Generate the next 100 samples based on the learned parameter
with model:
    posterior_predictive = pm.sample_posterior_predictive(trace, var_names=["theta"], random_seed=42)
    new_samples = np.random.binomial(n=1, p=posterior_predictive["theta"].mean(), size=100)

print("Generated samples:", new_samples)

# Opinion on prediction
print("The predictions align with the inferred bias parameter of the model.")


WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


AttributeError: `np.obj2sctype` was removed in the NumPy 2.0 release. Use `np.dtype(obj).type` instead.